<a href='https://ai.meng.duke.edu'> = <img align="left" style="padding-top:10px;" src=https://storage.googleapis.com/aipi_datasets/Duke-AIPI-Logo.png>

# Topic Modeling with Latent Dirichlet Allocation (LDA) and Non-Negative Matrix Factorization (NMF)
Topic modeling involves identifying the primary topic(s) for a text document or within a set of documents.  There are topic modeling approaches which use supervised learning to perform multi-class multi-label classification of documents, but this type of approach requires a large labeled training dataset, which we do not always have available.  In this notebook we will explore unsupervised topic modeling approaches using LDA and NMF.  The "topics" produced by unsupervised topic modeling techniques are actually clusters of similar words found in the document(s). The topic model discovers these word clusters based on the frequency of the words in each document and attempts to model what the topics might be and what each document's balance of topics is.

Topic modeling can be used to extract the main topic(s) from a single document or a collection of documents - in this notebook we will demonstrate topic modeling for a small collection of recent news articles.

In [205]:
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import itertools
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings("ignore")

## Get documents to tag with topics
We will use BeautifulSoup to get the content of a few articles from the web and strip the text content from the hmtl.

In [186]:
# Get article
article_urls = ['https://www.cbssports.com/college-basketball/news/duke-basketballs-game-vs-clemson-postponed-due-to-positive-covid-19-tests-in-blue-devils-program/',
                'https://www.cnbc.com/2021/04/26/apple-announces-1-billion-north-carolina-campus.html',
                'https://www.wral.com/coronavirus/100-of-duke-health-patients-in-icu-or-on-life-saving-treatment-are-unvaccinated-hospital-cases-rising/20045587/',
                'https://www.macworld.com/article/561815/apple-2022-ar-headset-homepod-mac.html',
                'https://www.dukechronicle.com/article/2021/12/duke-university-classes-online-spring-2022-semester-move-in-entry-testing-covid-vaccine-booster-mandate-require']
article_text = []
for url in article_urls:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # Extract body text from article
    bodytext = soup.find_all('p')
    bodytext = [i.text for i in bodytext]
    bodytext = ' '.join(bodytext)
    article_text.append(bodytext)


## Create features using Word Counts or TF-IDF
Before we can apply a topic model on our set of documents, we must first convert each document into a numeric feature vector.  We can use count vectorization or TF-IDF to do this.

In [206]:
def vectorize(documents, vectorizer_type='count'):
    # Use both 1-grams and 2-grams
    n_gram_range = (1, 2)

    if vectorizer_type == 'count':
        vectorizer = CountVectorizer(max_df=0.6,ngram_range=n_gram_range,
                                    stop_words=stopwords.words('english'))
        feature_vecs = vectorizer.fit_transform(documents)
        feature_vecs = feature_vecs.todense().tolist()
    else:
        vectorizer = TfidfVectorizer(max_df=0.6,ngram_range=n_gram_range,
                                    stop_words=stopwords.words('english'))
        feature_vecs = vectorizer.fit_transform(documents)
        feature_vecs = feature_vecs.todense().tolist()
        
    return feature_vecs, vectorizer

## Model topics using LDA or NMF
Now that our documents are represented by numeric feature vectors, we can apply our topic model.  When fitting the model, we need to select the number of topics we wish to include in our final list, which will correspond to the top n components extracted by LDA/NMF.

Each topic is represented by a cluster of similar keywords.  When printing our topic we can also select how many keywords we want to include to represent each topic.

In [207]:
def model_topics(vectorized_text,vectorizer,n_topics,n_words,model_type='lda'):
    if model_type=='lda':
        # Perform LDA 
        model = LatentDirichletAllocation(n_components=n_topics,learning_method='online', random_state=1)
        model.fit_transform(vectorized_text)
    else:
        model = NMF(n_components=n_topics,max_iter=500)
        model.fit_transform(vectorized_text)

    # Get the main keywords and scores corresponding to each topic
    vocab = vectorizer.get_feature_names_out()
    topics = []
    for comp in model.components_:
        # Get the top keywords for each topic
        sorted_words = [vocab[score] for score in np.argsort(comp)[::-1]][:n_words]
        # Get the scores for each top keyword
        sorted_scores = np.sort(comp)[::-1][:n_words]
        words_scores = zip(sorted_words,sorted_scores)
        topics.append(words_scores)

    return topics

In [251]:
feature_vecs, vectorizer = vectorize(article_text,vectorizer_type='tfidf')
topics = model_topics(feature_vecs,vectorizer,n_topics=5,n_words=4,model_type='lda')
for i,topic in enumerate(topics):
    print('Topic {} keywords: {}'.format(i,[word[0] for word in topic]))


Topic 0 keywords: ['apple', 'covid 19', '19', 'health']
Topic 1 keywords: ['apple', '000', 'engineering', 'campus']
Topic 2 keywords: ['wild', 'proportional', 'exec', 'jan asked']
Topic 3 keywords: ['cbs', 'sports', 'cbs sports', 'positive']
Topic 4 keywords: ['jan', 'students', 'test', 'email']


## Post-process topics
We may want to do some post-processing on the topics which our topic model identifies.  For example, we may have cases where a 1-gram is included twice in our keyword list for a topic - once by itself and then again as part of a 2-gram.  We may choose to remove the 1-gram if the word is included in a 2-gram since the two-gram may be more descriptive (e.g. "random" and "random forest").  We may also want to check and make sure we don't have any numeric-only keywords listed.

In [239]:
def dedupe_topics(topics,n_words):
    newtopics = []
    wordsused = []
    for topic in topics:
        newkeywords = []
        keywords = [word[0] for word in topic]
        for word in keywords:
            # Remove words that contain only digits
            if word.isdigit():
                continue
            # Remove 1-gram if word is included in a 2-gram
            elif sum([word in x for x in keywords])==1:
                newkeywords.append(word)
        newtopics.append(newkeywords[:n_words])
    return newtopics

In [250]:
feature_vecs, vectorizer = vectorize(article_text,vectorizer_type='tfidf')
topics = model_topics(feature_vecs,vectorizer,n_topics=5,n_words=15,model_type='lda')
deduped_topics = dedupe_topics(topics,n_words=4)
for i,keywords in enumerate(deduped_topics):
    print('Topic {} keywords: {}'.format(i,keywords))


Topic 0 keywords: ['apple', 'covid 19', 'health', 'year']
Topic 1 keywords: ['apple', 'engineering', 'campus', 'plans']
Topic 2 keywords: ['wild', 'proportional', 'exec', 'jan asked']
Topic 3 keywords: ['cbs sports', 'positive', 'players', 'josh']
Topic 4 keywords: ['students', 'email', 'must', 'drive']
